In [0]:
from typing import Optional

import fugue_bigquery.api as fbqa
import pandas as pd

from fugue_notebook import setup
from statsforecast import StatsForecast
from statsforecast.distributed.fugue import FugueBackend
from statsforecast.models import *

setup(is_lab=True)

/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.9/site-packages/fugue_sql/__init__.py:8: UserWarning: fsql and FugueSQLWorkflow now should be imported directly from fugue, fugue_sql will be removed in 0.9.0
  warnings.warn(
/databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py:171: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  original_result = python_builtin_import(name, globals, locals, fromlist, level)


In [0]:
# schema: *
def fill_missing_values(df: pd.DataFrame, max_date: str, min_date: Optional[str] = None) -> pd.DataFrame:
    min_ds = df['ds'].iloc[0]
    filled_df = pd.DataFrame({
        'unique_id': df['unique_id'].iloc[0],
        'ds': pd.date_range(min_ds if min_date is None else min_date, max_date, freq='D')
    })
    filled_df['ds'] = filled_df['ds'].astype(str)
    df['ds'] = df['ds'].astype(str)
    filled_df = filled_df.merge(df, how='left', on=['unique_id', 'ds'])
    filled_df['y'] = filled_df['y'].fillna(0)
    return filled_df

In [0]:
%%fsql spark
CONNECT bigquery SELECT 
    start_station_name AS unique_id,
    date AS ds,
    num_trips AS y
FROM `fuguedatabricks.bqml_tutorial.nyc_citibike_time_series`
WHERE date < '2016-06-01'

TRANSFORM PREPARTITION BY unique_id PRESORT ds ASC USING fill_missing_values(max_date='2016-05-31')
PERSIST
YIELD DATAFRAME AS data

/databricks/spark/python/pyspark/sql/pandas/conversion.py:626: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  [(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]


In [0]:
backend = FugueBackend(spark, {'fugue.spark.use_pandas_udf': True}, persist=True)

In [0]:
sf = StatsForecast(
   models = [ 
       Naive(), 
       MSTL(season_length=7, trend_forecaster=AutoETS(model='ZZN')),
       AutoETS(season_length=7),
       AutoCES(season_length=7),
       SeasonalNaive(season_length=7),
   ], # models to employ
   freq = 'D', # frequency per https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases
   backend = backend, # specify backend, if you want to run your pipeline locally, just remove it
   fallback_model = SeasonalNaive(season_length=7)
)
y_pred = sf.forecast(
    df = data.native.repartition(512, 'unique_id'), 
    h = 7
)

In [0]:
# Training was executed using 
#CREATE OR REPLACE MODEL bqml_tutorial.nyc_citibike_arima_model_default
#OPTIONS
#  (model_type = 'ARIMA_PLUS',
#   time_series_timestamp_col = 'date',
#   time_series_data_col = 'num_trips',
#   time_series_id_col = 'start_station_name',
#   DATA_FREQUENCY = 'DAILY'
#  ) AS
#SELECT *
#FROM bqml_tutorial.nyc_citibike_time_series
#WHERE date < '2016-06-01'

forecast_bigquery_sql = """
SELECT 
  start_station_name AS unique_id,
  forecast_timestamp AS ds,
  forecast_value AS BigQuery
FROM 
ML.FORECAST(
  MODEL `fuguedatabricks.bqml_tutorial.nyc_citibike_arima_model_default`, 
  STRUCT(7 AS horizon)
)
"""
forecast_bigquery = fbqa.load_sql(forecast_bigquery_sql, parallelism=1000, engine=spark).cache()

In [0]:
from datasetsforecast.losses import mae, rmse

In [0]:
# schema: unique_id:str,metric:str,Naive:float,MSTL:float,AutoETS:float,CES:float,SeasonalNaive:float,BigQuery:double
def evaluate(df: pd.DataFrame) -> pd.DataFrame:
    eval_ = {}
    models = df.loc[:, ~df.columns.str.contains('unique_id|^y|ds')].columns
    for model in models:
        eval_[model] = {}
        for metric in [mae, rmse]:
            eval_[model][metric.__name__] = metric(df['y'], df[model])
    eval_df = pd.DataFrame(eval_).rename_axis('metric').reset_index()
    eval_df.insert(0, 'unique_id', df['unique_id'].iloc[0])
    return eval_df

In [0]:
%%fsql spark

y_real = CONNECT bigquery SELECT 
            start_station_name AS unique_id,
            date AS ds,
            num_trips AS y
        FROM `fuguedatabricks.bqml_tutorial.nyc_citibike_time_series`
        WHERE date BETWEEN DATE("2016-06-01") AND DATE("2016-06-07")
        TRANSFORM PREPARTITION BY unique_id PRESORT ds ASC USING fill_missing_values(min_date='2016-06-01', 
                                                                                     max_date='2016-06-07')
        
SELECT 
    y_pred.*,
    y, 
    COALESCE(fb.BigQuery, 0) AS BigQuery
FROM y_real
LEFT JOIN y_pred 
ON y_pred.unique_id = y_real.unique_id AND y_pred.ds = y_real.ds
LEFT JOIN forecast_bigquery AS fb
ON y_pred.unique_id = fb.unique_id AND y_pred.ds = fb.ds

SELECT *
TRANSFORM PREPARTITION BY unique_id PRESORT ds ASC USING evaluate

SELECT 
    metric,
    AVG(Naive) AS Naive, 
    AVG(MSTL) AS MSTL, 
    AVG(AutoETS) AS AutoETS,
    AVG(CES) AS CES,
    AVG(SeasonalNaive) AS SeasonalNaive,
    AVG(BigQuery) AS BigQuery
GROUP BY metric

PERSIST
YIELD DATAFRAME AS evaluation
PRINT

/databricks/spark/python/pyspark/sql/pandas/conversion.py:626: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  [(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]
/databricks/spark/python/pyspark/sql/pandas/conversion.py:626: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  [(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]


,metric:str,Naive:double,MSTL:double,AutoETS:double,CES:double,SeasonalNaive:double,BigQuery:double
0,mae,28.949813,20.964255,22.464852,22.710158,24.690243,24.135331
1,rmse,37.282606,25.987708,27.031592,27.163590,33.158694,29.163746


In [0]:
evaluation.native.toPandas().set_index('metric').T.sort_values('rmse')

metric,rmse,mae
MSTL,25.987708,20.964255
AutoETS,27.031592,22.464852
CES,27.163590,22.710158
BigQuery,29.163746,24.135331
SeasonalNaive,33.158694,24.690243
Naive,37.282606,28.949813
